In [1]:
# Comparing ALL models at once
import os
import torch
import torch.nn as nn
import gradio as gr
import numpy as np
import joblib
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

# ======================================================
# 1. Define MultiHeadDistilBert
# ======================================================
class MultiHeadDistilBert(nn.Module):
    def __init__(self, base_name, num_labels_sent, num_labels_emot):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(base_name)
        hidden = 768
        self.dropout = nn.Dropout(0.2)
        self.classifier_sent = nn.Linear(hidden, num_labels_sent)
        self.classifier_emot = nn.Linear(hidden, num_labels_emot)

    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls = out.last_hidden_state[:, 0]
        cls = self.dropout(cls)
        return {
            "logits_sent": self.classifier_sent(cls),
            "logits_emot": self.classifier_emot(cls)
        }

# ======================================================
# 2. Load models
# ======================================================
models_dir = "./models"

# --- Logistic Regression + SVM ---
try:
    logreg_model = joblib.load(os.path.join(models_dir, "logreg_model.pkl"))
    svm_model = joblib.load(os.path.join(models_dir, "svm_model.pkl"))
    vectorizer = joblib.load(os.path.join(models_dir, "tfidf_vectorizer.pkl"))
    has_classical = True
    print("✅ Classical models loaded successfully!")
except Exception as e:
    print("⚠️ Skipping classical models:", e)
    has_classical = False
    logreg_model = svm_model = vectorizer = None

# --- DistilBERT ---
try:
    distil_dir = "./distilbert_ardor_saved"
    distil_tokenizer = AutoTokenizer.from_pretrained(distil_dir)
    distil_model = MultiHeadDistilBert("distilbert-base-uncased", 2, 6)
    distil_model.load_state_dict(torch.load(f"{distil_dir}/pytorch_model.bin", map_location="cpu"))
    distil_model.eval()
    has_distil = True
    print("✅ DistilBERT loaded successfully!")
except Exception as e:
    print("⚠️ Skipping DistilBERT:", e)
    has_distil = False

# --- RoBERTa ---
try:
    roberta_dir = os.path.join(models_dir, "roberta_ardor")
    roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_dir)
    roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_dir)
    roberta_model.eval()
    has_roberta = True
    print("✅ RoBERTa loaded successfully!")
except Exception as e:
    print("⚠️ Skipping RoBERTa:", e)
    has_roberta = False

# ======================================================
# 3. Label maps
# ======================================================
id2label_sent = {0: "Negative", 1: "Positive"}
id2label_emot = {0: "joy", 1: "sad", 2: "anger", 3: "fear", 4: "love", 5: "surprise"}

def softmax_temp(x, T=1.5):
    return torch.nn.functional.softmax(x / T, dim=-1)

# ======================================================
# 4. Prediction helpers
# ======================================================
def predict_classical(model, text):
    X = vectorizer.transform([text])
    probs = model.predict_proba(X)[0]
    idx = np.argmax(probs)
    label = "Positive" if idx == 1 else "Negative"
    conf = float(probs[idx])
    return label, conf, None

def predict_distilbert(text):
    inputs = distil_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        out = distil_model(**inputs)
        ps = softmax_temp(out["logits_sent"]).cpu().numpy()[0]
        pe = softmax_temp(out["logits_emot"]).cpu().numpy()[0]
    sent_idx = int(np.argmax(ps))
    sent_label = id2label_sent[sent_idx]
    sent_conf = float(ps[sent_idx])
    top_idx = np.argsort(-pe)[:3]
    emotions = [(id2label_emot[i], float(pe[i])) for i in top_idx]
    return sent_label, sent_conf, emotions

def predict_roberta(text):
    inputs = roberta_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = roberta_model(**inputs).logits
        ps = softmax_temp(logits).cpu().numpy()[0]
    idx = int(np.argmax(ps))
    label = id2label_sent[idx]
    conf = float(ps[idx])
    return label, conf, None

# ======================================================
# 5. Compare all models at once
# ======================================================
def compare_models(text):
    text = text.strip()
    if not text:
        return "⚠️ Please enter some text to analyze."

    results = []
    # Classical models
    if has_classical:
        for name, model in [("Logistic Regression", logreg_model), ("SVM", svm_model)]:
            label, conf, _ = predict_classical(model, text)
            results.append(f"**{name}** → {label} ({conf*100:.1f}%)")

    # DistilBERT
    if has_distil:
        label, conf, emo = predict_distilbert(text)
        emo_str = " | ".join([f"{e[0]}: {e[1]:.2f}" for e in emo])
        results.append(f"**DistilBERT** → {label} ({conf*100:.1f}%)\nTop Emotions: {emo_str}")

    # RoBERTa
    if has_roberta:
        label, conf, _ = predict_roberta(text)
        results.append(f"**RoBERTa** → {label} ({conf*100:.1f}%)")

    if not results:
        return "⚠️ No models available. Please ensure they’re trained and saved."
    
    return "\n\n".join(results)

# ======================================================
# 6. Gradio Interface
# ======================================================
iface = gr.Interface(
    fn=compare_models,
    inputs=gr.Textbox(label="Enter text to analyze", placeholder="Type something like: I love this movie!"),
    outputs=gr.Markdown(label="Model Comparison Results"),
    title="🎭 The Ardor Scale – Sentiment & Emotion Comparison",
    description=(
        "Enter a sentence to see predictions from all models:\n"
        "• Logistic Regression and SVM (TF-IDF baselines)\n"
        "• DistilBERT (sentiment + emotion)\n"
        "• RoBERTa (advanced transformer)"
    ),
    examples=[
        ["I absolutely love this project!"],
        ["This is terrible and makes me so upset."],
        ["It’s okay, nothing special."],
        ["I'm so excited for the concert tonight!"]
    ],
    allow_flagging="never"
)

# ======================================================
# 7. Launch App
# ======================================================
if __name__ == "__main__":
    iface.launch(share=True)


/Users/sabasiddiqi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


⚠️ Skipping classical models: [Errno 2] No such file or directory: './models/logreg_model.pkl'
✅ DistilBERT loaded successfully!
⚠️ Skipping RoBERTa: Repo id must be in the form 'repo_name' or 'namespace/repo_name': './models/roberta_ardor'. Use `repo_type` argument if needed.
Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://4937448eb0f988df2f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
